In [1]:
import chromadb, requests
from pprint import pprint
from models import ChromaDbResult

In [2]:
text = "penalties in the GDPR"
resp = requests.post(
        "http://localhost:8001/embeddings",
        json={"text": text},
        timeout=20
    )
data = resp.json()
emb = data.get("embedding", [])
print(type(emb[0]))

<class 'float'>


In [3]:

chroma_client = chromadb.HttpClient(host="localhost", port=8000)
col = chroma_client.get_collection(name="gdpr")
raw = col.query(query_embeddings=[emb], n_results=2)
print(type(raw))
pprint(raw)

<class 'dict'>
{'data': None,
 'distances': [[0.8643079, 0.8943157]],
 'documents': [[' (152) Where this Regulation does not harmonise '
                'administrative penalties or where necessary in other cases, '
                'for example in cases of serious infringements of this '
                'Regulation, Member States should implement a system which '
                'provides for effective, proportionate and dissuasive '
                'penalties. The nature of such penalties, criminal or '
                'administrative, should be determined by Member State law.',
                ' (148) In order to strengthen the enforcement of the rules of '
                'this Regulation, penalties including administrative fines '
                'should be imposed for any infringement of this Regulation, in '
                'addition to, or instead of appropriate measures imposed by '
                'the supervisory authority pursuant to this Regulation. In a '
                '

In [6]:
results = [ChromaDbResult(
        id=raw["ids"][0][i],
        document=raw["documents"][0][i], 
        metadata=raw["metadatas"][0][i],
        distance=raw["distances"][0][i]
        ) for i in range(len(raw["ids"][0]))]
print(results)

[ChromaDbResult(id='668080e7-fc12-4087-b04f-fab9cfb9b0c1', document=' (152) Where this Regulation does not harmonise administrative penalties or where necessary in other cases, for example in cases of serious infringements of this Regulation, Member States should implement a system which provides for effective, proportionate and dissuasive penalties. The nature of such penalties, criminal or administrative, should be determined by Member State law.', metadata={'article_title': '', 'part': 'preamble', 'chapter_title': '', 'article': 0, 'section': 0, 'section_title': '', 'chapter': 0}, distance=0.8643079), ChromaDbResult(id='e661cc5a-09b7-4764-a023-b1cccfc2c468', document=' (148) In order to strengthen the enforcement of the rules of this Regulation, penalties including administrative fines should be imposed for any infringement of this Regulation, in addition to, or instead of appropriate measures imposed by the supervisory authority pursuant to this Regulation. In a case of a minor inf